In [1]:
import os
os.chdir('/home/megatron/work')

import megatron
import pandas as pd
import numpy as np
import sqlite3

In [2]:
P = megatron.load_pipeline('/home/megatron/work/test.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/home/megatron/work/test.pkl'

In [2]:
# example using pandas
generator = False
lahman_file = 'data/lahman_csv/core/Batting.csv'
lahman = pd.read_csv(lahman_file)
exclude = ['playerID','yearID','stint','teamID','lgID']

if generator:
    lahman_generator = megatron.io.generator.PandasGenerator(lahman, 1000,
                                                             exclude_cols=exclude)
else:
    lahman_data = megatron.io.dataset.PandasData(lahman, exclude_cols=exclude)
    
inputs = megatron.nodes.from_dataframe(lahman, exclude_cols=exclude)

In [ ]:
# example using csv
generator = True
lahman_file = 'data/lahman_csv/core/Batting.csv'
exclude = ['playerID','yearID','stint','teamID','lgID']

if generator:
    lahman_generator = megatron.io.generator.CSVGenerator(lahman_file, 1000,
                                                          exclude_cols=exclude)
else:
    lahman_data = megatron.io.dataset.CSVData(lahman_file, exclude_cols=exclude)
    
inputs = megatron.nodes.from_csv(lahman_file, exclude_cols=exclude, eager=False)

In [ ]:
# example using sql database

# make sql database from csv
lahman_df = pd.read_csv('data/lahman_csv/core/Batting.csv')
conn = sqlite3.connect('lahman')
conn.execute('DROP TABLE IF EXISTS batting')
lahman_df.to_sql('batting', conn, index=False)

generator = True
conn = sqlite3.connect('lahman')
query = 'SELECT * FROM batting'

if generator:
    lahman_generator = megatron.io.generator.SQLGenerator(conn, query, 1000)
else:
    lahman_data = megatron.io.dataset.SQLData(conn, query)
    
inputs = megatron.nodes.from_sql(conn, query, eager=False)

In [3]:
fillna = megatron.layers.Impute({np.nan: 0})
inputs = fillna(inputs)
inputs = megatron.layers.Cast(np.int)(inputs)

# helpers
def single_fn(h, d, t, hr):
    return h - d - t - hr
singles = megatron.layers.Lambda(single_fn)(inputs[['H','2B','3B','HR']], 'Singles')
hit_types = megatron.layers.Concatenate()([singles]+inputs[['2B','3B','HR']], 'hit_types')
TB = megatron.layers.Dot(W=np.array([1,2,3,4]))(hit_types, 'TB')

# basics
PA = megatron.layers.Add()(inputs[['AB', 'BB', 'HBP', 'SH', 'SF']], 'PA')
BBp = megatron.layers.Divide()([inputs['BB'], PA], 'BBpct')
Kp = megatron.layers.Divide()([inputs['SO'], PA], 'Kpct')
def obp(h, bb, hbp, ab, sf):
    return megatron.helpers.safe_divide(h + bb + hbp, ab + bb + hbp + sf)
OBP = megatron.layers.Lambda(obp)(inputs[['H','BB','HBP','AB','SF']], 'OBP')
SLG = megatron.layers.Divide()([TB, inputs['AB']], 'SLG')
AVG = megatron.layers.Divide()(inputs[['H', 'AB']], 'AVG')
ISO = megatron.layers.Subtract()([SLG, AVG], 'ISO')
def babip(h, hr, ab, k, sf):
    return megatron.helpers.safe_divide(h - hr, ab - k - hr + sf)
BABIP = megatron.layers.Lambda(babip)(inputs[['H','HR','AB','SO','SF']], 'BABIP')

outputs = [PA, BBp, Kp, OBP, SLG, AVG, ISO, BABIP]

outputs = megatron.nodes.FeatureSet(outputs)
outputs = megatron.layers.Lambda(np.round, decimals=2)(outputs)

storage_db = sqlite3.connect('data/megatron_default.db')
P = megatron.Pipeline(inputs, outputs, name='test', version=0.1, storage_db=storage_db)

In [4]:
if generator:
    P.fit_generator(lahman_generator)
    out = P.transform_generator(lahman_generator, cache_result=True)
else:
    P.fit(lahman_data)
    out = P.transform(lahman_data, cache_result=True)

In [ ]:
P.save('/home/megatron/work/')

In [ ]:
stored = P.storage.read(lookup=['0', '1'])
stored

In [ ]:
megatron.visuals.pipeline_imsave(P, 'img/sabermetrics.png')
megatron.visuals.pipeline_imshow(P)